[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/samsoe/matrix/blob/master/notebooks/soil_moisture_collection.ipynb#scrollTo=pxCW8QKdC8SS)

This notebook utilizes 6 .xls files as source data.  This source data was downloaded from the 'Decagon' machine managed by MPG Ranch.  The 6 .xls files contain data from moisture probes collocated with weather stations on the ranch.  
* [Link to hosted xls files](https://drive.google.com/open?id=1PSQAjHN_RCMSRkYYeNDLu7WpL6hzt0Y2)

# Load Tools

In [0]:
import pandas as pd
import numpy as np

# Source Data
* [Link to hosted xls source files](https://drive.google.com/open?id=1PSQAjHN_RCMSRkYYeNDLu7WpL6hzt0Y2)

In [0]:
stations_data = [
    {'weather': 'KMTFLORE5', 'soil': 'sainfoin bench', 'soil_data': 'SB-10Jun2019-0844.xls', 'short_name': 'sb' 
    , 'url': 'https://drive.google.com/uc?id=1lKUITDnHd7zSPILsCPFYACBoiWn92KfE'},
    {'weather': 'KMTFLORE4', 'soil': 'baldy draw', 'soil_data': 'BD-10Jun2019-0844.xls', 'short_name': 'bd'
    , 'url': 'https://drive.google.com/uc?id=1cg6zActqzs8fEmqC30JbM-EgBjBdg7xV'},
    {'weather': 'KMTFLORE6', 'soil': 'baldy summit', 'soil_data': 'BS-07Jun2019-1554.xls', 'short_name': 'bs'
    , 'url': 'https://drive.google.com/uc?id=1bQlHOP4F-knljrtbk3l7tCoI-aU9EIBu'},
    {'weather': 'KMTFLORE3', 'soil': 'indian ridge', 'soil_data': 'IR-10Jun2019-0845.xls', 'short_name': 'ir'
    , 'url': 'https://drive.google.com/uc?id=1NjKOTYIFTrf36Z7-RaERHacwUosbme02'},
    {'weather': 'KMTFLORE7', 'soil': 'south baldy ridge', 'soil_data': 'SBR-10Jun2019-0845.xls', 'short_name': 'sbr'
    , 'url': 'https://drive.google.com/uc?id=1_Rrkl-8Yr3DxWi6VKgu5eC6jheP65iox'},
    {'weather': 'KMTFLORE7', 'soil': 'orchard house', 'soil_data': 'OH-10Jun2019-0846.xls', 'short_name': 'oh'
    , 'url': 'https://drive.google.com/uc?id=1edvoRGUPKeIGFw9Y-_EvylDJ0BcMwxtH'}
]

# Load and Restructure Downloaded Data

In [0]:
def reshape_data(src, station):
  # load data
  df = pd.read_excel(src, header=[0,2])
  
  # restructure multi index
  df = df.set_index(df[src]['Measurement Time']).drop((src, 'Measurement Time'), axis=1)
  df = df.unstack().unstack(level=1).reset_index(level=1).reset_index().set_index('Measurement Time')

  # rename columns
  df.columns = ['port', 'kPa Potential', '°C Temp']
  
  # intitalize depth column
  df['depth (in)'] = ''

  # label moisture probe depth for each port
  for index, row in df.iterrows():
    if row.port == 'Port 1' or row.port == 'Port 3':
      row['depth (in)'] = 6
    elif row.port == 'Port 2' or row.port == 'Port 4':
      row['depth (in)'] = 36
      
  # station id label
  df['station'] = station
  
  return df

In [0]:
# array to hold individual station dataframes
df_stations = []

# loop through stations_data dictionary to load and restructure station data
for station in stations_data:
    df_stations.append({station['short_name']: reshape_data(station['soil_data'], station['soil'])})

# Combine Station DataFrames

In [0]:
stations_combined = pd.DataFrame()

for station in df_stations:
  for key, val in station.items():
    stations_combined = pd.concat([stations_combined, station[key]])

# Clean

In [0]:
# Port 5 is not configured, remove it
stations_combined = stations_combined.loc[stations_combined.port != 'Port 5']

In [0]:
stations_combined.shape

(171296, 5)

In [0]:
# The default for NA in the downloaded datasets is '***', change it to 
stations_combined = stations_combined.replace(to_replace = "***", value = np.NaN)

In [0]:
stations_combined = stations_combined.dropna()

In [0]:
stations_combined.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 160153 entries, 2000-11-07 06:00:00 to 2019-06-09 18:00:00
Data columns (total 5 columns):
port             160153 non-null object
kPa Potential    160153 non-null float64
°C Temp          160153 non-null float64
depth (in)       160153 non-null int64
station          160153 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 7.3+ MB


In [0]:
# reshape to single measurement per row
stations_combined = pd.melt(stations_combined.reset_index(),\
        id_vars=["Measurement Time",  "station", "port", "depth (in)"],\
        var_name="measurement",\
        value_name="value")

# Overview

### Unique Stations

In [0]:
stations_combined.station.unique().tolist()


['sainfoin bench',
 'baldy draw',
 'baldy summit',
 'indian ridge',
 'south baldy ridge',
 'orchard house']

In [0]:
stations_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320306 entries, 0 to 320305
Data columns (total 6 columns):
Measurement Time    320306 non-null datetime64[ns]
station             320306 non-null object
port                320306 non-null object
depth (in)          320306 non-null int64
measurement         320306 non-null object
value               320306 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 14.7+ MB


### Unique Ports

In [0]:
stations_combined.port.unique()

array(['Port 1', 'Port 2', 'Port 3', 'Port 4'], dtype=object)

### Measurement Depths

In [0]:
stations_combined['depth (in)'].unique()

array([ 6, 36])

### Measurement Types

In [0]:
stations_combined.measurement.unique()

array(['kPa Potential', '°C Temp'], dtype=object)

# Export combined stations 

In [0]:
stations_combined.to_csv('soil_moisture-190611.csv', index=False)

'soil_moisture-190611.csv' output can be downloaded here: [link](https://drive.google.com/open?id=1F-OzrbaqUR28skpxc8Ai6hjpSq_VjSpD)